In [ ]:
import polars as pl
from pandas.core.interchange.dataframe_protocol import DataFrame

# Login using e.g. `huggingface-cli login` to access this dataset
df = pl.read_parquet('hf://datasets/lmsys/lmsys-chat-1m/data/train-*.parquet')

In [100]:
df2 = df.filter(df["language"] == "English")
df3 = (df2
       .filter(df2["conversation"].list.len() == 22)
       .filter(pl.col("conversation").is_not_null()).filter(pl.col("conversation").list.eval(
    # For each element, check if it has valid struct fields
    pl.element().struct.field("content").is_not_null() &
    pl.element().struct.field("role").is_not_null() &
    # Ensure the fields are strings
    pl.element().struct.field("content").cast(pl.String).is_not_null() &
    pl.element().struct.field("role").cast(pl.String).is_not_null()
).list.all())
       .limit(100))

In [101]:
df4 = pl.DataFrame(df3["conversation"])

In [106]:


df5 = df4.with_columns(
    pl.when(
        pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 7).list.last().struct.field(
            "role") != "user"
    ).then(
        pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 7).list.slice(0, pl.col(
            "conversation").list.len() - 1)
    ).otherwise(
        pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 7)
    )
)


In [108]:
import json

# Convert the nested conversation column to JSON strings
csv_ready_df = df5.with_columns(
    pl.col("conversation").map_elements(
        lambda row_list: json.dumps([
            {"content": item["content"], "role": item["role"]}
            for item in row_list
        ]), return_dtype=pl.String
    ).alias("conversation")
)  # Drop the original nested column and write to CSV
csv_ready_df.drop("conversation").write_csv("lmsys.csv")